# 제2 고지 : 자연스러운  코드로 
## STEP 22 : 연산자 오버로드(3)

![image](../assets/%ED%91%9C%2022-1.png)

이전 단계에서는 `*`,`+` 연산자를 지원하도록 확장했는데, 이외에도 많은 연산자가 존재한다. 따라서 위의 표와 같이 추가 연산자들을 구현한다.  
(물론, 위의 표 이외에도 `a//b`, `a%b` ,`a+=1`,`a-=2` 와 같은 연산자들도 있지만, 현재는 자주 사용하자는 연산자만 구현한다)

1. 음수 (부호변환)
2. 뺄셈
3. 나눗셈
4. 거듭제곱

### 22.1 음수 (부호변환)

음수의 미분은 $y=-x$ 일때 $\frac{\partial y}{\partial x }=-1$ 이다. 따라서 역전파는 출력에서 전해지는 미분에 $-1$ 을 곱하여 하류로 보내주면 된다.


In [7]:
import weakref
import numpy as np 

def as_variable(obj):
    if isinstance(obj,Variable):
        return obj 
    return Variable(obj)


class Variable:
    __array_priority__ = 200 
    def __init__(self, data, name=None):
        if data is not None:
            if not isinstance(data, np.ndarray):
                raise TypeError("{} is not supported".format(type(data)))

        self.data = data
        self.name = name  # 변수 구분을 위한 `이름` 설정
        self.grad = None
        self.creator = None
        self.generation = 0

    @property
    def ndim(self):
        return self.data.ndim

    @property
    def shape(self):
        return self.data.shape

    @property
    def size(self):
        return self.data.size

    @property
    def dtype(self):
        return self.data.dtype

    def __len__(self):
        return len(self.data)

    def __repr__(self):
        if self.data is None:
            return "variable(None)"
        p = str(self.data).replace("\n", "\n" + " " * 9)
        return "variable(" + p + ")"

    def set_creator(self, func):
        self.creator = func
        self.generation = func.generation + 1

    def cleargrad(self):
        self.grad = None

    def backward(self, retain_grad=False):  # `retain_grad` 추가
        if self.grad is None:
            self.grad = np.ones_like(self.data)

        funcs = []
        seen_set = set()

        def add_func(f):
            if f not in seen_set:
                funcs.append(f)
                seen_set.add(f)
                funcs.sort(key=lambda x: x.generation)

        add_func(self.creator)

        while funcs:
            f = funcs.pop()
            gys = [output().grad for output in f.outputs]  # output is weakref
            gxs = f.backward(*gys)
            if not isinstance(gxs, tuple):
                gxs = (gxs,)

            for x, gx in zip(f.inputs, gxs):
                if x.grad is None:
                    x.grad = gx
                else:
                    x.grad = x.grad + gx

                if x.creator is not None:
                    add_func(x.creator)

            if not retain_grad:
                for y in f.outputs:
                    y().grad = None  # y is weakref이기 때문에 y()로 호출

    def __mul__(self, other):
        return mul(self, other)

    def __rmul__(self, other):
        return mul(self, other)

    def __add__(self, other):
        return add(self, other)

    def __radd__(self, other):
        return add(self, other)


def as_array(x):
    if np.isscalar(x):
        return np.array(x)
    return x

class Function:
    def __call__(self, *inputs):
        ################################
        inputs = [as_variable(x) for x in inputs]
        ################################
        xs = [x.data for x in inputs]
        ys = self.forward(*xs)
        if not isinstance(ys, tuple):
            ys = (ys,)
        outputs = [Variable(as_array(y)) for y in ys]

        self.generation = max([x.generation for x in inputs])
        for output in outputs:
            output.set_creator(self)
        self.inputs = inputs
        self.outputs = [weakref.ref(output) for output in outputs]
        return outputs if len(outputs) > 1 else outputs[0]

    def forward(self, xs):
        raise NotImplementedError()

    def backward(self, gys):
        raise NotImplementedError()

class Add(Function):
    def forward(self, x0,x1):
        y = x0 + x1
        return y
    def backward(self, gy):
        # 역전파시 , 입력이 1개 , 출력이 2개 
        return gy,gy 
    
def add(x0,x1):
    x1 = as_array(x1)
    return Add()(x0,x1)



class Mul(Function):
    def forward(self, x0, x1):
        y = x0 * x1
        return y

    def backward(self, gy):
        x0, x1 = self.inputs[0].data, self.inputs[1].data
        return gy * x1, gy * x0
    
def mul(x0,x1):
    x1= as_array(x1)
    return Mul()(x0,x1)

In [8]:
class Neg(Function):
    def forward(self,x):
        return -x 
    def backward(self, gy):
        return -gy 
    
def neg(x):
    return Neg()(x)

Variable.__neg__=neg

In [9]:
x = Variable(np.array(2.0))
y = -x 
print(y)

variable(-2.0)


### 22.2 뺄셈

뺄셈의 미분은 $y=x_0-x_1$일 때 $\begin{bmatrix} \frac{\partial y}{\partial x_0} & \frac{\partial y}{\partial x_1} \end{bmatrix} =  \begin{bmatrix} 1 & -1 \end{bmatrix}$ 이다.  
따라서 역전파는 출력에서 전해지는 미분값에 $1$을 곱한값이 $x_0$의 미분결과가 되고, $-1$을 곱한 값이 $x_1$ 의 미분결과가 된다.  


In [10]:
class Sub(Function):
    def forward(self,x0,x1):
        y = x0-x1 
        return y 
    
    def backward(self, gy):
        return gy,-gy 
    
def sub(x0,x1):
    x1 = as_array(x1)
    return Sub()(x0,x1)

Variable.__sub__=sub

In [11]:
x = Variable(np.array(2.0))
y = x+3.0
print(y)

variable(5.0)


여기서 중요한 것은 위의 코드는 `y=x0-x1` 계산은 수행할 수 있지만, `x0`이 `Variable` 인스턴스가 아닌 `y=2.0-x` 와 같은 코드 처리는 어렵다.  
그 이유는 `2.0`의의`__sub__` 를 호출하려 했으나, 정의되어 있지 않아 `x`의 `__rsub__` 를 호출하려 했지만 이 역시 정의되어 있지 않기 때문이다.  
그래서 다음과 같이 추가적으로 구현한다. 
 

In [12]:
    
def rsub(x0,x1):
    x1 = as_array(x1)
    return Sub()(x1,x0) # x0 와 x1의 순서를 바꾼다.

Variable.__rsub__=rsub

In [13]:
x = Variable(np.array(2.0))
y1 = 2.0 - x 
y2 = x - 1.0
print(y1)
print(y2)

variable(0.0)
variable(1.0)


### 22.3 나눗셈

나눗셈의 미분은 $y=\frac{x_0}{x_1}$ 일때, $\begin{bmatrix} \frac{\partial y}{\partial x_0} & \frac{\partial y}{\partial x_1} \end{bmatrix} =  \begin{bmatrix} \frac{1}{x_1} & \frac{-x_0}{x_1^2} \end{bmatrix}$ 이다.   
나눗셈 역시 뺄셈과 마찬가지로 좌/우항의 순서가 중요하므로,동일한 아이디어로 `__rdiv__` 뿐만 아니라 `__rtruediv__` 도 구현한다. 


In [14]:
class Div(Function):
    def forward(self,x0,x1):
        y = x0/x1 
        return y 
    
    def backward(self, gy):
        x0,x1 = self.inputs[0].data,self.inputs[1].data 
        gx0 = gy / x1 
        gx1 = gy * (-x0 / x1 ** 2)
        return gx0,gx1
    
def div(x0,x1):
    x1 = as_array(x1)
    return Div()(x0,x1)

def rdiv(x0,x1):
    x1 = as_array(x1)
    return Div()(x1,x0) # x0 와 x1의 순서를 바꾼다.

Variable.__truediv__=div
Variable.__rtruediv__=div

In [16]:
x = Variable(np.array(2.0))
y1 = 1 / x 
y2 = x / 4
print(y1)
print(y2)

variable(2.0)
variable(0.5)


### 22.4 거듭제곱

거듭제곱은 $y=x^c$ 형태로 표현된다. 이때 $\frac{\partial y}{\partial x} = cx^{c-1}$ 이다. 여기서 $\frac{\partial y}{\partial c}$ 의 값도 구할 수 있겠지만, 실전에서는 거의 사용하지 않으므로 제외한다. 


In [17]:
class Pow(Function):
    def __init__(self,c) :
        self.c= c
    def forward(self,x):
        y = x ** self.c 
        return y
    
    def backward(self, gy):
        x = self.inputs[0].data 
        c = self.c 
        gx = c * x ** (c-1) * gy 
        return gx 
    
    
def pow(x,c):
    return Pow(c)(x)


Variable.__pow__=pow

In [18]:
x = Variable(np.array(2.0))
y = x **3
print(y)

variable(8.0)
